In [37]:
import socket
from keras.models import load_model
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from PIL import Image
import os

#TCP Client
if __name__ == '__main__':
    client_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
    client_sock.connect(('127.0.0.1', 26001))
    # Connect and receive file size info from server
    while True:
        file_info = client_sock.recv(1024)
        file_size = int.from_bytes(file_info, byteorder='little')
        received_data = b""
        print(file_size) #데이터 크기 출력
        while True:
            print("Receiving data...")
            data = client_sock.recv(file_size)
            received_data += data
            if len(received_data) >= file_size:
                break
        save_path = 'C:/Users/Aiot/jupiter/sample_test/c.png'
        with open(save_path, 'wb') as file:
            file.write(received_data)
        print(f"File saved: {save_path}")
        #-------------------------------------------- 여기까지 데이터 받고 파일생성
        #-------------------------------------------- 여기부터 검사
        model_path = 'C:/Users/Aiot/jupiter/test_model.hdf5' #모델 경로
        image_path = 'C:/Users/Aiot/jupiter/sample_test/c.png'
        try:
            model = tf.keras.models.load_model(model_path)
        except Exception as e:
            print("모델 로드 중 오류 발생:", e) #모델 로드 실패시 오류
        
        img_height = 240
        img_width = 320
        class_names = ['Bexel_A', 'Bexel_B', 'Duracell', 'Energizer', 'Rocket']
        #클래스 이름 명시적으로 안적으면 오류남 분명 학습할때 입력된건데 멍청한놈
        img = image.load_img(image_path, target_size=(img_height, img_width))
        img_array = image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  #이미지 불러오기 및 전처리
        predictions = model.predict(img_array) #실행 예측값 나옴
        score = tf.nn.softmax(predictions[0])  #예측값가지고 클래스에 맞냐 안맞냐? 구분하는 듯
        max_prob_index = np.argmax(score) #구분된거로 클래스 인덱스값 찾음
        max_prob_class = class_names[max_prob_index] #표현
        max_prob = np.round(100 * score[max_prob_index].numpy(), 2) #가장 높은 확률 찾음
        for i in range(len(class_names)): # 전체 클래스 확률 계산하는거라 없어도 됨
          print(f"{class_names[i]}: {np.round(100 * score[i].numpy(), 2)}%")
         # print()
        
        print(f"이미지 파일: {os.path.basename(image_path)}")
        print("예측:", max_prob_class)
        
        if max_prob_class == 'Bexel_A':
            print(1)   
            a = int(1)
            client_sock.send(a.to_bytes(4,byteorder='little'))
        elif max_prob_class == 'Bexel_B':
            print(2)
            a = int(2)
            client_sock.send(a.to_bytes(4,byteorder='little'))
        elif max_prob_class == 'Duracell':
            print(3)
            a = int(3)
            client_sock.send(a.to_bytes(4,byteorder='little'))
        elif max_prob_class == 'Energizer':
            a = int(4)
            client_sock.send(a.to_bytes(4,byteorder='little'))
            print(4)
        else:
            a = int(5)
            client_sock.send(a.to_bytes(4,byteorder='little'))
            print(5)

        

222827
Receiving data...
File saved: C:/Users/Aiot/jupiter/sample_test/c.png
1/1 [==============================] - 0s 86ms/step
Bexel_A: 18.42%
Bexel_B: 15.62%
Duracell: 15.5%
Energizer: 34.79%
Rocket: 15.67%
이미지 파일: c.png
예측: Energizer
4
224495
Receiving data...
File saved: C:/Users/Aiot/jupiter/sample_test/c.png
1/1 [==============================] - 0s 82ms/step
Bexel_A: 14.98%
Bexel_B: 40.17%
Duracell: 14.93%
Energizer: 14.92%
Rocket: 15.0%
이미지 파일: c.png
예측: Bexel_B
2
253365
Receiving data...
File saved: C:/Users/Aiot/jupiter/sample_test/c.png
1/1 [==============================] - 0s 83ms/step
Bexel_A: 15.14%
Bexel_B: 16.02%
Duracell: 15.15%
Energizer: 15.14%
Rocket: 38.55%
이미지 파일: c.png
예측: Rocket
5
236508
Receiving data...
File saved: C:/Users/Aiot/jupiter/sample_test/c.png
1/1 [==============================] - 0s 83ms/step
Bexel_A: 14.89%
Bexel_B: 14.89%
Duracell: 14.89%
Energizer: 14.89%
Rocket: 40.44%
이미지 파일: c.png
예측: Rocket
5


ConnectionResetError: [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다